# Rating and Ranking


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Example NFC North 2020-2021
| date       | day | season | neutral | playoff | hteam | ateam | hscore | ascore |
|------------|-----|--------|---------|---------|-------|-------|--------|--------|
| 9/13/2020  | 4   | 2020   | 0       | 0       | MIN   | GB    | 34     | 43     |
| 9/13/2020  | 4   | 2020   | 0       | 0       | DET   | CHI   | 23     | 27     |
| 9/20/2020  | 11  | 2020   | 0       | 0       | GB    | DET   | 42     | 21     |
| 11/1/2020  | 53  | 2020   | 0       | 0       | GB    | MIN   | 22     | 28     |
| 11/8/2020  | 60  | 2020   | 0       | 0       | MIN   | DET   | 34     | 20     |
| 11/16/2020 | 68  | 2020   | 0       | 0       | CHI   | MIN   | 13     | 19     |
| 11/29/2020 | 81  | 2020   | 0       | 0       | GB    | CHI   | 41     | 25     |
| 12/6/2020  | 88  | 2020   | 0       | 0       | CHI   | DET   | 30     | 34     |
| 12/13/2020 | 95  | 2020   | 0       | 0       | DET   | GB    | 24     | 31     |
| 12/20/2020 | 102 | 2020   | 0       | 0       | MIN   | CHI   | 27     | 33     |
| 1/3/2021   | 116 | 2020   | 0       | 0       | DET   | MIN   | 35     | 37     |
| 1/3/2021   | 116 | 2020   | 0       | 0       | CHI   | GB    | 16     | 35     |

In [33]:
import csv
games = []
with open('../nfcnorth202021.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_NONE)
    header=next(reader,None)
    print(header[5:])
    for game in reader:
        print(game[5:])
        games.append(game[5:])



['hteam', 'ateam', 'hscore', 'ascore']
['MIN', 'GB', '34', '43']
['DET', 'CHI', '23', '27']
['GB', 'DET', '42', '21']
['GB', 'MIN', '22', '28']
['MIN', 'DET', '34', '20']
['CHI', 'MIN', '13', '19']
['GB', 'CHI', '41', '25']
['CHI', 'DET', '30', '34']
['DET', 'GB', '24', '31']
['MIN', 'CHI', '27', '33']
['DET', 'MIN', '35', '37']
['CHI', 'GB', '16', '35']


In [18]:
games[1][0]

'DET'

In [39]:
numgames = len(games)
scale = 400
K = 30

names = ['CHI','DET','GB','MIN']
elo = [1000,1000,1000,1000]

for i in range(numgames-1,0,-1):
    hteam = games[i][0]
    ateam = games[i][1]
    hscore = games[i][2]
    ascore = games[i][3]
    
    #get ratings
    hrat = elo[names.index(hteam)]
    arat = elo[names.index(ateam)]
    diff = hrat - arat
    
    #calc expected results/probs of winning
    hexp = 1./(1. + 10**(diff/scale))
    aexp = 1. - hexp
    
    #update ratings
    if (hscore > ascore): #home team win
        elo[names.index(hteam)] += K*(1. - hexp)
        elo[names.index(ateam)] += K*(0. - aexp)
    elif (ascore > hscore): #away team win
        elo[names.index(hteam)] += K*(0. - hexp)
        elo[names.index(ateam)] += K*(1. - aexp)
    #no else or draws needed
    
    #[print(name,elo[names.index(name)]) for name in names ]
[print(name,elo[names.index(name)]) for name in names ];

CHI 966.7354262235857
DET 932.8438376750813
GB 1051.782820850352
MIN 1048.637915250981


## Warnings/Cautions

### Campbell's Law
The more any quantitative social indicator is used for social decision-making, the more subject it will be to corruption pressures and the more apt it will be to distort and corrupt the social processes it is intended to monitor.

### Goodhart's Law
When a measure becomes a target, it ceases to be a good measure.

### spectral ranking


In [40]:
L = np.array([
    [0,1./3.,1./3.,1./2.],
    [0.5,0,1./3.,0.],
    [0.5,1./3.,0,0.5],
    [0,1./3.,1./3.,0.]
])

v,w = np.linalg.eig(L)

print(v)
print(w[:,0]/w[3,0])

[ 1.        +0.j         -0.33333333+0.23570226j -0.33333333-0.23570226j
 -0.33333333+0.j        ]
[1.5   -0.j 1.3125-0.j 1.6875-0.j 1.    -0.j]


### Colley for NFC North example


In [79]:
Cmat = np.zeros((4,4))
bvec = np.zeros((4,1))

for i in range(0,4):
    Cmat[i,i] = 2.
    bvec[i] = 1.
    

numgames = len(games)

names = ['CHI','DET','GB','MIN']

for i in range(0,numgames,1):
    hteam = games[i][0]
    ateam = games[i][1]
    hscore = games[i][2]
    ascore = games[i][3]
    hidx = names.index(hteam)
    aidx = names.index(ateam)
    
    Cmat[hidx,hidx] += 1.  #add 1 to total games played by home team
    Cmat[aidx,aidx] += 1.
    
    Cmat[hidx,aidx] += -1. #add -1 to offdiagonal entry corresponding to opponent
    Cmat[aidx,hidx] += -1.
    
    if (hscore > ascore): #home team win
        bvec[hidx] += 1./2.
        bvec[aidx] += -1/2.
    elif (ascore > hscore): #away team win
        bvec[aidx] += 1./2.
        bvec[hidx] += -1/2.

Crating = np.linalg.solve(Cmat,bvec)



print(Cmat)
print(bvec)

print('Team:  Colley rating')
for i in range(0,4):
    print(f"{names[i]}: {Crating[i][0]:4.2f}")





[[ 8. -2. -2. -2.]
 [-2.  8. -2. -2.]
 [-2. -2.  8. -2.]
 [-2. -2. -2.  8.]]
[[ 0.]
 [-1.]
 [ 3.]
 [ 2.]]
Team:  Colley rating
CHI: 0.40
DET: 0.30
GB: 0.70
MIN: 0.60


### 2016 Big Ten CBB


In [59]:
b1gnames = ['Illinois','Michigan','Maryland','Michigan State','Iowa','Ohio State','Northwestern',
           'Rutgers','Indiana','Nebraska','Minnesota','Purdue','Penn State','Wisconsin']

tmat = np.zeros((14,14))

tmat[0,:] = np.array([0,0,0,0,0,0,0,2,0,0,2,1,0,0])
tmat[1,:] = np.array([1,0,1,0,0,0,1,1,0,1,2,1,2,0])
tmat[2,:] = np.array([1,1,0,0,1,2,2,1,0,1,0,1,1,1])
tmat[3,:] = np.array([1,1,1,0,0,2,1,2,1,0,1,0,2,1])
tmat[4,:] = np.array([1,2,0,2,0,0,1,1,0,1,1,2,1,0])
tmat[5,:] = np.array([2,1,0,0,1,0,2,2,0,1,1,0,1,0])
tmat[6,:] = np.array([1,0,0,0,0,0,0,1,0,2,2,0,1,1])
tmat[7,:] = np.array([0,0,0,0,0,0,0,0,0,0,1,0,0,0])
tmat[8,:] = np.array([2,1,1,0,2,1,1,1,0,2,2,1,0,1])
tmat[9,:] = np.array([1,0,0,1,0,0,0,2,0,0,1,0,1,0])
tmat[10,:] = np.array([0,0,1,0,0,0,0,1,0,0,0,0,0,0])
tmat[11,:] = np.array([0,1,1,1,0,1,1,1,0,2,1,0,1,2])
tmat[12,:] = np.array([1,0,0,0,1,0,1,1,1,1,1,0,0,0])
tmat[13,:] = np.array([2,1,1,1,1,1,0,1,1,1,1,0,1,0])

print(tmat)

b1gwins = np.zeros(14)
for i in range(0,14):
    b1gwins[i] = tmat[i,:].sum()
    
print("Team: Wins")
for i in range(0,14):
    print(f"{b1gnames[i]}: {b1gwins[i]}")
    

[[0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 2. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 2. 1. 2. 0.]
 [1. 1. 0. 0. 1. 2. 2. 1. 0. 1. 0. 1. 1. 1.]
 [1. 1. 1. 0. 0. 2. 1. 2. 1. 0. 1. 0. 2. 1.]
 [1. 2. 0. 2. 0. 0. 1. 1. 0. 1. 1. 2. 1. 0.]
 [2. 1. 0. 0. 1. 0. 2. 2. 0. 1. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 2. 2. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [2. 1. 1. 0. 2. 1. 1. 1. 0. 2. 2. 1. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 2. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 1. 1. 0. 2. 1. 0. 1. 2.]
 [1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0.]
 [2. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0.]]
Team: Wins
Illinois: 5.0
Michigan: 10.0
Maryland: 12.0
Michigan State: 13.0
Iowa: 12.0
Ohio State: 11.0
Northwestern: 8.0
Rutgers: 1.0
Indiana: 15.0
Nebraska: 6.0
Minnesota: 2.0
Purdue: 12.0
Penn State: 7.0
Wisconsin: 12.0


In [80]:
t2 = np.dot(tmat,tmat)

print(t2)


b1g2wins = np.zeros(14)
for i in range(0,14):
    b1g2wins[i] = t2[i,:].sum()

print("Team: Second Order Wins")
for i in range(0,14):
    print(f"{b1gnames[i]}: {b1g2wins[i]}")

[[ 0.  1.  3.  1.  0.  1.  1.  3.  0.  2.  3.  0.  1.  2.]
 [ 5.  2.  3.  2.  3.  3.  5. 11.  2.  7.  9.  2.  4.  4.]
 [12.  6.  3.  5.  4.  2.  8. 15.  2. 12. 16.  4. 10.  4.]
 [13.  5.  4.  1.  8.  4. 10. 14.  3. 11. 15.  4.  7.  3.]
 [ 7.  4.  7.  3.  1.  6.  7. 15.  3.  9. 15.  3. 12.  7.]
 [ 6.  2.  2.  3.  1.  0.  3. 12.  1.  7. 15.  5.  6.  2.]
 [ 5.  1.  3.  3.  2.  1.  1. 10.  2.  2.  7.  1.  3.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [11.  8.  5.  8.  3.  4.  8. 19.  1. 10. 16.  8. 11.  4.]
 [ 2.  1.  2.  0.  1.  2.  2.  6.  2.  1.  6.  1.  2.  1.]
 [ 1.  1.  0.  0.  1.  2.  2.  1.  0.  1.  1.  1.  1.  1.]
 [13.  5.  5.  4.  5.  6.  7. 15.  4.  8. 12.  2. 11.  3.]
 [ 5.  3.  2.  3.  2.  1.  2.  8.  0.  5.  9.  4.  3.  2.]
 [10.  6.  4.  3.  5.  5.  9. 16.  2.  7. 14.  7.  8.  3.]]
Team: Second Order Wins
Illinois: 18.0
Michigan: 62.0
Maryland: 103.0
Michigan State: 102.0
Iowa: 99.0
Ohio State: 65.0
Northwestern: 41.0
Rutgers: 2.0
Indiana: 116.0
Nebras

In [78]:
b1gL = np.copy(tmat)

for j in range(0,14):
    b1gL[:,j] = b1gL[:,j]/b1gL[:,j].sum()

v,w = np.linalg.eig(b1gL)


b1gspecrat = np.real(w[:,0]/max(w[:,0]))

print("Team: Spectral Rating")
for i in range(0,14):
    #print(f"{b1gnames[i]}: {b1gspecrat[i]}")
    print("{}: {:5.1f}".format(b1gnames[i],b1gspecrat[i]))

Team: Spectral Rating
Illinois:  20.5
Michigan:  56.5
Maryland:  95.6
Michigan State: 115.5
Iowa: 112.1
Ohio State:  46.0
Northwestern:  35.9
Rutgers:   1.0
Indiana: 120.2
Nebraska:  32.0
Minnesota:  16.0
Purdue: 110.3
Penn State:  67.6
Wisconsin: 124.5
